Imports

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import STL
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

np.random.seed(42)

Load in the data

In [2]:
weather = pd.read_csv('./data/new-new-york-weather.csv')
weather.shape

C:\Users\deval\AppData\Local\Temp\ipykernel_4476\1420603750.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv('./data/new-new-york-weather.csv')


(368796, 15)

In [3]:
weather.dtypes

date                      object
temperature_2m_max        object
temperature_2m_min        object
daylight_duration         object
sunshine_duration         object
uv_index_max              object
uv_index_clear_sky_max    object
rain_sum                  object
showers_sum               object
snowfall_sum              object
precipitation_hours       object
wind_speed_10m_max        object
wind_gusts_10m_max        object
latitude                  object
longitude                 object
dtype: object

In [4]:
weather = weather.drop(columns = ['uv_index_max','uv_index_clear_sky_max'])

Drop strings in the dataframe that came from improper appending

In [5]:
weather[weather['daylight_duration'].astype(str).str.contains('day')]

,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
5084,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
65675,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
126266,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
187023,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
247614,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
308205,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude


In [6]:
weather = weather.drop(weather[weather['daylight_duration'].astype(str).str.contains('day')].index)

In [7]:
weather[weather['daylight_duration'].astype(str).str.contains('day')]

,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude


Sort the dataframe by date

In [8]:
weather = weather.sort_values('date')

Delete the time in all the dates and remove the white space made

In [9]:
weather['date'] = weather['date'].str.replace('05:00:00+00:00', '')
weather['date'] = weather['date'].str.strip()

In [10]:
weather.head()

,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude
291050,2005-01-01,6.0785,-1.6715,32634.35,20676.693,0.3,0.0,0.0,2.0,19.211996,46.8,43.1,-73.86
251630,2005-01-01,8.6855,-2.1145,32994.484,28458.79,0.2,0.0,0.0,2.0,23.73271,54.719997,42.1957,-78.6287
303460,2005-01-01,5.847,-2.803,32549.463,10120.855,0.2,0.0,0.0,1.0,17.873556,43.92,43.35,-73.42
260390,2005-01-01,7.4214997,-1.4285,32994.484,28465.23,0.1,0.0,0.0,1.0,19.67195,45.0,42.213,-74.982
260755,2005-01-01,5.9925,-2.6075,32994.484,28727.3,0.2,0.0,0.0,1.0,20.620804,47.88,42.1821,-74.9847


In [11]:
weather[['temperature_2m_max','temperature_2m_min',"daylight_duration","sunshine_duration","rain_sum",'showers_sum',
         "snowfall_sum","precipitation_hours","wind_speed_10m_max","wind_gusts_10m_max","latitude","longitude"]] = weather[['temperature_2m_max','temperature_2m_min',"daylight_duration","sunshine_duration","rain_sum",'showers_sum',
         "snowfall_sum","precipitation_hours","wind_speed_10m_max","wind_gusts_10m_max","latitude","longitude"]].apply(pd.to_numeric)

In [12]:
weather['precipitation_total'] = weather['rain_sum'] + weather['snowfall_sum'] 
weather['location'] = list(zip(weather['latitude'],weather['longitude']))

In [13]:
# Group by 'location'
grouped = weather.groupby('location')

# Drop duplicates based on 'date' for each group
weather = pd.concat([group.drop_duplicates(subset=['date']) for _, group in grouped], ignore_index=True)

# Display the first few rows of the cleaned DataFrame
weather.shape

(368790, 15)

In [14]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368790 entries, 0 to 368789
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   date                 368790 non-null  object 
 1   temperature_2m_max   368790 non-null  float64
 2   temperature_2m_min   368790 non-null  float64
 3   daylight_duration    368790 non-null  float64
 4   sunshine_duration    368788 non-null  float64
 5   rain_sum             368790 non-null  float64
 6   showers_sum          368790 non-null  float64
 7   snowfall_sum         368790 non-null  float64
 8   precipitation_hours  368790 non-null  float64
 9   wind_speed_10m_max   368790 non-null  float64
 10  wind_gusts_10m_max   368790 non-null  float64
 11  latitude             368790 non-null  float64
 12  longitude            368790 non-null  float64
 13  precipitation_total  368790 non-null  float64
 14  location             368790 non-null  object 
dtypes: float64(13), o

In [15]:
weather.isnull().sum()

date                   0
temperature_2m_max     0
temperature_2m_min     0
daylight_duration      0
sunshine_duration      2
rain_sum               0
showers_sum            0
snowfall_sum           0
precipitation_hours    0
wind_speed_10m_max     0
wind_gusts_10m_max     0
latitude               0
longitude              0
precipitation_total    0
location               0
dtype: int64

In [16]:
weather.dropna(inplace = True)

In [17]:
weather.head()

,date,temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,rain_sum,showers_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,latitude,longitude,precipitation_total,location
0,2005-01-01,13.779,0.779,33603.098,29108.1450,0.0,0.0,0.00,0.0,16.781560,34.920000,40.5795,-74.1502,0.00,"(40.5795, -74.1502)"
1,2005-01-02,6.779,-1.471,33650.117,18150.7850,0.0,0.0,0.00,0.0,13.910169,31.319998,40.5795,-74.1502,0.00,"(40.5795, -74.1502)"
2,2005-01-03,12.829,6.129,33700.723,4058.1921,5.5,0.0,0.00,7.0,11.525623,27.359999,40.5795,-74.1502,5.50,"(40.5795, -74.1502)"
3,2005-01-04,9.529,2.979,33754.844,6188.0500,2.6,0.0,0.00,7.0,13.320000,40.320000,40.5795,-74.1502,2.60,"(40.5795, -74.1502)"
4,2005-01-05,3.579,0.479,33812.383,0.0000,11.5,0.0,1.33,19.0,12.727922,25.919998,40.5795,-74.1502,12.83,"(40.5795, -74.1502)"


In [30]:
weather = weather.drop(columns = ['rain_sum', 'showers_sum', 'snowfall_sum'])

In [31]:
weather['date'] = pd.to_datetime(weather['date'])
weather = weather.sort_values(by = 'date')

Forecast each feature into 2030 with an ARIMA model

In [38]:
# Example function to preprocess data, aggregate to monthly, and forecast with AutoARIMA
def preprocess_and_forecast(data, forecast_steps, seasonal_period, train_end_date):
    """
    Preprocess and forecast weather data using monthly aggregation, STL, and AutoARIMA,
    with a train-test split.
    
    Args:
    - data: DataFrame, columns = ['date', 'latitude', 'longitude', 'feature1', ..., 'featureN']
    - forecast_steps: int, number of future periods to forecast (for testing and beyond)
    - seasonal_period: int, period of seasonality (e.g., 12 for monthly data)
    - train_end_date: str, cutoff date for training data (e.g., "2023-12")
    
    Returns:
    - train_data: ndarray, shape = (n_samples, train_timestamps, n_features)
    - test_data: ndarray, shape = (n_samples, test_timestamps, n_features)
    """
    # Step 1: Aggregate to monthly averages
    data['month'] = data['date'].dt.to_period('M')
    monthly_data = data.groupby(['latitude', 'longitude', 'month']).mean(numeric_only = True).reset_index()
    
    # Step 2: Pivot to create time series for each feature
    feature_columns = [col for col in monthly_data.columns if col not in ['latitude', 'longitude', 'month']]
    time_series_data = []
    for (lat, lon), group in monthly_data.groupby(['latitude', 'longitude']):
        series = group.set_index('month')[feature_columns].values
        time_series_data.append(series)
    time_series_data = np.array(time_series_data)  # (n_samples, n_timestamps, n_features)
    
    # Step 3: Split data into train and test sets
    n_timestamps = time_series_data.shape[1]
    n_train = len(pd.date_range(start='2005-01', end=train_end_date, freq='M'))
    n_test = n_timestamps - n_train
    train_data = time_series_data[:, :n_train, :]
    test_data = time_series_data[:, n_train:, :]
    
    # Step 4: Forecast with AutoARIMA
    forecasted_train_data = np.zeros((train_data.shape[0], n_train + forecast_steps, train_data.shape[2]))
    forecasted_test_data = np.zeros((test_data.shape[0], n_test + forecast_steps, test_data.shape[2]))
    
    for feature_idx in range(len(feature_columns)):
        for sample_idx in range(train_data.shape[0]):
            train_series = train_data[sample_idx, :, feature_idx]
            stl = STL(train_series, period=seasonal_period)
            result = stl.fit()
            detrended = train_series - result.trend
            
            # AutoARIMA forecasting
            model = auto_arima(detrended, seasonal=True, m=seasonal_period, stepwise=True, suppress_warnings=True)
            forecast = model.predict(n_periods=forecast_steps)
            trend_forecast = np.full(forecast_steps, result.trend[-1])
            full_forecast = trend_forecast + forecast
            
            # Combine historical and forecast data
            forecasted_train_data[sample_idx, :, feature_idx] = np.concatenate((train_series, full_forecast))
            forecasted_test_data[sample_idx, :, feature_idx] = np.concatenate((test_data[sample_idx, :, feature_idx], full_forecast))
    
    return forecasted_train_data, forecasted_test_data

In [39]:
forecast_steps = 12 * 6 + 1  # Forecast 6 years and 1 month into the future (73 months)
seasonal_period = 12  # Monthly seasonality
train_end_date = "2023-12"

train_data, test_data = preprocess_and_forecast(weather, forecast_steps, seasonal_period, train_end_date)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (166,) + inhomogeneous part.

Run a time-series k-means clustering model

In [ ]:
# Flatten data for clustering
train_data_flat = train_data.reshape(train_data.shape[0], -1)
test_data_flat = test_data.reshape(test_data.shape[0], -1)

# Define the pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10)),
    ("kmeans", TimeSeriesKMeans(n_clusters=2, metric="softdtw", n_jobs=12, verbose=True))
])

# Fit the pipeline on training data
pipeline.fit(train_data_flat)

# Transform and predict clusters for training and testing data
train_clusters = pipeline["kmeans"].predict(train_data_flat)
test_clusters = pipeline["kmeans"].predict(test_data_flat)

print("Train Clusters:", train_clusters)
print("Test Clusters:", test_clusters)